In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

from src.model import CLIP
from src.dataset import FlickrDataset, UnsplashDataset
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizer
from src.loss_function import symmetric_loss


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# dataset = UnsplashDataset(tokenizer, "../../data/unsplash/photos.tsv*")
dataset = FlickrDataset(image_folder_path = "../../data/flickr-dataset/Images/", caption_path = "../../data/flickr-dataset/captions.txt")
train_loader = DataLoader(dataset, batch_size = 4, shuffle=True)    
img, label = next(iter(train_loader))
label = tokenizer(label, padding=True, truncation=True, max_length = 76, return_tensors="pt")

cfg = {'model':{"projections": 768}}
clip_model = CLIP(cfg)
img_emb, txt_emb, logits = clip_model(img, label)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
loss = symmetric_loss(img_emb, txt_emb, logits)
loss.backward()